# SSDの学習と検証
- 本ファイルでは、SSDの学習と検証の実施を行います。手元のマシンで動作を確認後、AWSのGPUマシンで計算します。
- p2.xlargeで約6時間かかります。

## Library

In [2]:
# パッケージのimport
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [3]:
# 乱数シードを設定
def something_seed(seed=1234):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

seed = 1234
something_seed(seed)

In [4]:
# cpu or gpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('使用デバイス：　', device)

使用デバイス：　 cuda:0


## DataLoaderの作成

In [5]:
from utils.ssd_model import make_datapath_list, VOCDataset, DataTransform, Anno_xml2list, od_collate_fn

# ファイルパスのリストを取得
rootpath = './data/VOCdevkit/VOC2012/'
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

# Datasetを作成
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 
               'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 
               'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)   # BGRの平均値
input_size = 300  # 画像のサイズを300x300にする

train_dataset = VOCDataset(train_img_list, train_anno_list, phase='train', transform=DataTransform(input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))
val_dataset = VOCDataset(val_img_list, val_anno_list, phase='val', transform=DataTransform(input_size, color_mean), transform_anno=Anno_xml2list(voc_classes))

# DataLoaderを作成
batch_size = 32
train_dataloader = data.DataLoader(train_dataset, batch_size, shuffle=True, collate_fn=od_collate_fn)  # 画像毎に物体検出数が異なるため
val_dataloader = data.DataLoader(val_dataset, batch_size, shuffle=False, collate_fn=od_collate_fn)

# 辞書オブジェクトへ
dataloaders_dict = {
    'train' : train_dataloader,
    'val' : val_dataloader
}

## ネットワークモデルの作成

In [6]:
from utils.ssd_model import SSD

# SSD300の設定
ssd_cfg = {
    'num_classes' : 21,                                                                   #　背景クラスを含めた合計クラス数
    'input_size' : 300,                                                                     # 画像の入力サイズ
    'bbox_aspect_num' : [4, 6, 6, 6, 4, 4],                                  # 出力するDBoxのアスペクト比の種類
    'feature_maps' : [38, 19, 10, 5, 3, 1],                                   # 各source（feature map）の画像サイズ　
    'steps' : [8, 16, 32, 64, 100, 300],                                        # DBOXの大きさを決める
    'min_sizes' : [30, 60, 111, 162, 213, 264],                         # 
    'max_sizes' : [60, 111, 162, 213, 264, 315],
    'aspect_ratios' : [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}

# ネットワークモデル
net = SSD(phase='train', cfg=ssd_cfg)

# SSDの重みの初期値を設定
vgg_weights = torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)  # vgg忘れに注意　　つけないと全てのネットワークに対して重み入れようとしてエラーに

# ssdのvgg以外のnetworkの重みはHeの初期値（sqrt(2/n)）で初期化
# Heの初期値は画像でReLUを活性化カンスに使うときに使用する
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:  # bias項がある場合
            nn.init.constant_(m.bias, 0.0)
            
# Heの重みを適用
net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)
# applyのこういう使い方かっこいい

# GPUが使えるかを確認
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('使用デバイス：　', device)

print('ネットワーク設定完了：　学習済みの重みをロードしました')

使用デバイス：　 cuda:0
ネットワーク設定完了：　学習済みの重みをロードしました


In [7]:
# SSDモデルの確認
# self,vgg, self.extrasというふうにインスタンス変数でModuleListをもっている
net.vgg

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
  (17): Conv2d(256, 512, kernel_siz

In [8]:
net.extras

ModuleList(
  (0): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1))
  (3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (4): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
  (6): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
)

In [9]:
net.loc

ModuleList(
  (0): Conv2d(512, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [10]:
net.conf

ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [11]:
net

SSD(
  (vgg): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

## 損失関数、最適化手法の設定


In [12]:
from utils.ssd_model import MultiBoxLoss

# 損失関数の設定
criterion = MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3, device=device)

# 最適化手法の設定
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)


## 学習と検証
モデルを学習させる関数を作成する。kerasで言うところのfitみたいなもの

In [22]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    # GPUが使えるか
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('使用デバイス：　', device)
    
    # ネットワークをGPUへ
    net.to(device)
    
    # ネットワークがある程度固定できれば高速化させる
    torch.backends.cudnn.benchmark = True
    
    # イテレーションカウンタをセット
    iteration = 1
    epoch_train_loss = 0.0   # epochの損失和
    epoch_val_loss = 0.0      # epochの損失和
    logs = []
    
    # epochのループ
    # この中で trainとvalを行う
    for epoch in range(num_epochs+1):
        # 開始時刻を保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        
        print('-------------------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------------------')
        
        #  epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
                print('  (train)  ')
            else:
                # 検証は１０学習に付き１回行う
                if ((epoch+1)%10) == 0:
                    net.eval()
                    print('-------------------------')
                    print('  (val)  ')
                else:
                    continue
                    
            # データローダーからminibatchずつ取り出すグループ
            for images, targets in dataloaders_dict[phase]:   # train, valどっちから取り出すのか指定
                # GPUに送る
                images = images.to(device)
                targets = [anno.to(device) for anno in targets]  # リストの各要素のテンソルをGPUへ
                # そのまま送ると不都合あるのかな？
                
                # optimizerを初期化
                optimizer.zero_grad()
                
                # 順伝播計算
                with torch.set_grad_enabled(phase=='train'):
                    outputs = net(images)
                    loss_l, loss_c = criterion(outputs, targets)
                    loss = loss_l + loss_c
                    
                    # 学習時
                    if phase == 'train':
                        loss.backward()  # 勾配求まる
                        
                        # 勾配が大きくなりすぎると学習が不安定になるのでclipで最大でも２にする
                        nn.utils.clip_grad_value_(net.parameters(), clip_value=2.0)
                        optimizer.step()
                        
                        # 10回に１回lossを表示
                        if (iteration%10 == 0):
                            t_iter_finish =  time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(iteration, loss.item(), duration))
                            t_iter_start = time.time()   # iterの時間計測再開
                            
                        epoch_train_loss += loss.item()
                        iteration += 1
                        
                    # 検証時
                    else:
                        epoch_val_loss += loss
                        
        # epochのphaseごとのlossと正解率　　　表示させたら０．０に戻す
        t_epoch_finish =  time.time()
        print('-------------------------')
        print('epoch {} || EPOCH_TRAIN_Loss : {:.4f} || EPOCH_VAL_Loss : {:.4f}'.format(
            epoch+1, epoch_train_loss, epoch_val_loss))
        print('timer : {:.4f} sec'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()  # epochの時間計測再開
        
        # logを保存
        log_epoch = {'epoch': epoch+1, 
                    'train_loss': epoch_train_loss,
                    'val_loss' : epoch_val_loss}
        logs.append(log_epoch)   # リストにdictを追加していく
        df = pd.DataFrame(logs)
        df.to_csv('log_output.csv')  
        # 毎epoch、log保存していったら結構な数になるじゃないか？　-> 
        # ずっと同じlog_output.csvに保存しているから、フィアルが増えるわけではなくて上書きされてく
        
        # 保存し終わったところでlossを初期化
        epoch_train_loss = 0.0   # epochの損失和
        epoch_val_loss = 0.0      # epochの損失和
        
        # ネットワークを保存する  全部で５回
        if ((epoch+1)%10 == 0):
            torch.save(net.state_dict(), 'weights/ssd300_'+str(epoch+1)+'.pth')
            
        

In [23]:
# 学習・検証をする
# これはaws上で回す
num_epochs = 50
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)

使用デバイス：　 cuda:0
-------------------------
Epoch 1/50
-------------------------
  (train)  
イテレーション 10 || Loss: 16.2334 || 10iter: 26.7792 sec.
イテレーション 20 || Loss: 11.7053 || 10iter: 26.9027 sec.
イテレーション 30 || Loss: 9.7830 || 10iter: 26.0329 sec.
イテレーション 40 || Loss: 8.5813 || 10iter: 26.0299 sec.
イテレーション 50 || Loss: 7.8798 || 10iter: 25.3848 sec.
イテレーション 60 || Loss: 7.6992 || 10iter: 25.4548 sec.
イテレーション 70 || Loss: 8.2362 || 10iter: 25.9629 sec.
イテレーション 80 || Loss: 7.6965 || 10iter: 25.6545 sec.
イテレーション 90 || Loss: 7.9131 || 10iter: 26.9423 sec.
イテレーション 100 || Loss: 7.4269 || 10iter: 25.7277 sec.
イテレーション 110 || Loss: 8.3546 || 10iter: 25.8601 sec.
イテレーション 120 || Loss: 7.9093 || 10iter: 25.8468 sec.
イテレーション 130 || Loss: 8.5538 || 10iter: 25.8708 sec.
イテレーション 140 || Loss: 7.5315 || 10iter: 25.7175 sec.
イテレーション 150 || Loss: 7.9638 || 10iter: 25.9866 sec.
イテレーション 160 || Loss: 7.3287 || 10iter: 26.3172 sec.
イテレーション 170 || Loss: 7.1616 || 10iter: 26.4569 sec.
-------------------------
epoch 

/home/ubuntu/learning_pytorch/pytorch_advanced/2_objectdetection/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)


イテレーション 180 || Loss: 6.7914 || 10iter: 1.1985 sec.
イテレーション 190 || Loss: 7.4061 || 10iter: 25.5719 sec.
イテレーション 200 || Loss: 7.1740 || 10iter: 24.7703 sec.
イテレーション 210 || Loss: 7.6191 || 10iter: 24.9231 sec.
イテレーション 220 || Loss: 6.8582 || 10iter: 25.0482 sec.
イテレーション 230 || Loss: 7.2173 || 10iter: 24.6704 sec.
イテレーション 240 || Loss: 6.9910 || 10iter: 25.1692 sec.
イテレーション 250 || Loss: 6.7796 || 10iter: 24.9119 sec.
イテレーション 260 || Loss: 7.0567 || 10iter: 25.6398 sec.
イテレーション 270 || Loss: 6.7054 || 10iter: 25.1808 sec.
イテレーション 280 || Loss: 6.7140 || 10iter: 25.2004 sec.
イテレーション 290 || Loss: 6.8483 || 10iter: 24.9504 sec.
イテレーション 300 || Loss: 6.6562 || 10iter: 25.0302 sec.
イテレーション 310 || Loss: 7.0647 || 10iter: 25.6449 sec.
イテレーション 320 || Loss: 7.0994 || 10iter: 24.6448 sec.
イテレーション 330 || Loss: 6.1300 || 10iter: 25.0118 sec.
イテレーション 340 || Loss: 6.8270 || 10iter: 25.1270 sec.
イテレーション 350 || Loss: 6.6457 || 10iter: 25.1836 sec.
-------------------------
epoch 2 || EPOCH_TRAIN_Loss : 1265.6008

イテレーション 1500 || Loss: 4.8555 || 10iter: 26.5537 sec.
イテレーション 1510 || Loss: 4.9337 || 10iter: 32.3723 sec.
イテレーション 1520 || Loss: 4.7666 || 10iter: 31.7331 sec.
イテレーション 1530 || Loss: 4.8957 || 10iter: 29.0082 sec.
イテレーション 1540 || Loss: 4.9651 || 10iter: 27.6294 sec.
イテレーション 1550 || Loss: 4.9612 || 10iter: 25.4703 sec.
イテレーション 1560 || Loss: 4.8556 || 10iter: 25.1583 sec.
イテレーション 1570 || Loss: 5.1933 || 10iter: 25.2790 sec.
イテレーション 1580 || Loss: 4.9792 || 10iter: 25.0757 sec.
イテレーション 1590 || Loss: 4.9387 || 10iter: 25.3893 sec.
イテレーション 1600 || Loss: 5.4220 || 10iter: 25.2626 sec.
イテレーション 1610 || Loss: 5.3618 || 10iter: 25.3255 sec.
-------------------------
epoch 9 || EPOCH_TRAIN_Loss : 939.8295 || EPOCH_VAL_Loss : 0.0000
timer : 499.6820 sec
-------------------------
Epoch 10/50
-------------------------
  (train)  
イテレーション 1620 || Loss: 4.6523 || 10iter: 22.3816 sec.
イテレーション 1630 || Loss: 5.4023 || 10iter: 24.9205 sec.
イテレーション 1640 || Loss: 5.4367 || 10iter: 25.5741 sec.
イテレーション 1650 || 

イテレーション 2790 || Loss: 4.9136 || 10iter: 25.2559 sec.
イテレーション 2800 || Loss: 4.8950 || 10iter: 25.2554 sec.
イテレーション 2810 || Loss: 4.8202 || 10iter: 25.5695 sec.
イテレーション 2820 || Loss: 4.7070 || 10iter: 25.3231 sec.
イテレーション 2830 || Loss: 4.6911 || 10iter: 25.1630 sec.
イテレーション 2840 || Loss: 4.3908 || 10iter: 25.1089 sec.
イテレーション 2850 || Loss: 5.1614 || 10iter: 25.4998 sec.
イテレーション 2860 || Loss: 4.2445 || 10iter: 24.8736 sec.
-------------------------
epoch 16 || EPOCH_TRAIN_Loss : 816.8035 || EPOCH_VAL_Loss : 0.0000
timer : 475.1047 sec
-------------------------
Epoch 17/50
-------------------------
  (train)  
イテレーション 2870 || Loss: 4.4413 || 10iter: 14.6408 sec.
イテレーション 2880 || Loss: 4.4199 || 10iter: 25.0775 sec.
イテレーション 2890 || Loss: 5.0881 || 10iter: 24.9833 sec.
イテレーション 2900 || Loss: 4.0498 || 10iter: 25.4200 sec.
イテレーション 2910 || Loss: 4.9200 || 10iter: 25.9115 sec.
イテレーション 2920 || Loss: 4.6946 || 10iter: 25.3200 sec.
イテレーション 2930 || Loss: 4.5353 || 10iter: 25.3147 sec.
イテレーション 2940 ||

イテレーション 4080 || Loss: 4.2268 || 10iter: 25.0435 sec.
イテレーション 4090 || Loss: 3.8061 || 10iter: 24.9413 sec.
イテレーション 4100 || Loss: 4.2081 || 10iter: 25.1473 sec.
イテレーション 4110 || Loss: 4.6559 || 10iter: 25.7820 sec.
-------------------------
epoch 23 || EPOCH_TRAIN_Loss : 742.6849 || EPOCH_VAL_Loss : 0.0000
timer : 481.3762 sec
-------------------------
Epoch 24/50
-------------------------
  (train)  
イテレーション 4120 || Loss: 4.3575 || 10iter: 6.7521 sec.
イテレーション 4130 || Loss: 3.7403 || 10iter: 25.2201 sec.
イテレーション 4140 || Loss: 3.8410 || 10iter: 24.8763 sec.
イテレーション 4150 || Loss: 3.9683 || 10iter: 24.7639 sec.
イテレーション 4160 || Loss: 3.6520 || 10iter: 24.8387 sec.
イテレーション 4170 || Loss: 4.6073 || 10iter: 25.8163 sec.
イテレーション 4180 || Loss: 4.1499 || 10iter: 25.1204 sec.
イテレーション 4190 || Loss: 3.7670 || 10iter: 24.8711 sec.
イテレーション 4200 || Loss: 3.4465 || 10iter: 25.1465 sec.
イテレーション 4210 || Loss: 4.2926 || 10iter: 24.8799 sec.
イテレーション 4220 || Loss: 4.1729 || 10iter: 25.2509 sec.
イテレーション 4230 || 

-------------------------
epoch 30 || EPOCH_TRAIN_Loss : 697.0999 || EPOCH_VAL_Loss : 736.0670
timer : 648.1093 sec
-------------------------
Epoch 31/50
-------------------------
  (train)  
イテレーション 5380 || Loss: 3.6679 || 10iter: 24.8729 sec.
イテレーション 5390 || Loss: 3.3618 || 10iter: 24.8731 sec.
イテレーション 5400 || Loss: 3.3553 || 10iter: 24.8745 sec.
イテレーション 5410 || Loss: 3.7257 || 10iter: 25.0114 sec.
イテレーション 5420 || Loss: 3.7018 || 10iter: 25.2251 sec.
イテレーション 5430 || Loss: 3.7273 || 10iter: 24.9822 sec.
イテレーション 5440 || Loss: 4.1567 || 10iter: 25.1148 sec.
イテレーション 5450 || Loss: 4.3756 || 10iter: 25.2039 sec.
イテレーション 5460 || Loss: 4.3127 || 10iter: 24.7942 sec.
イテレーション 5470 || Loss: 4.0371 || 10iter: 24.8165 sec.
イテレーション 5480 || Loss: 3.5204 || 10iter: 25.0262 sec.
イテレーション 5490 || Loss: 4.0212 || 10iter: 25.7812 sec.
イテレーション 5500 || Loss: 3.7448 || 10iter: 25.2664 sec.
イテレーション 5510 || Loss: 3.5276 || 10iter: 25.3904 sec.
イテレーション 5520 || Loss: 3.6274 || 10iter: 24.8090 sec.
イテレーション 5530 

イテレーション 6640 || Loss: 3.3893 || 10iter: 24.9375 sec.
イテレーション 6650 || Loss: 3.4997 || 10iter: 25.5568 sec.
イテレーション 6660 || Loss: 3.1764 || 10iter: 25.1814 sec.
イテレーション 6670 || Loss: 4.1085 || 10iter: 25.1801 sec.
イテレーション 6680 || Loss: 3.5051 || 10iter: 25.5460 sec.
イテレーション 6690 || Loss: 3.4768 || 10iter: 25.4877 sec.
イテレーション 6700 || Loss: 3.3892 || 10iter: 25.5286 sec.
イテレーション 6710 || Loss: 3.6534 || 10iter: 25.2998 sec.
イテレーション 6720 || Loss: 3.7752 || 10iter: 24.7650 sec.
イテレーション 6730 || Loss: 4.0835 || 10iter: 25.3597 sec.
イテレーション 6740 || Loss: 4.0249 || 10iter: 25.2911 sec.
イテレーション 6750 || Loss: 3.5418 || 10iter: 25.2868 sec.
イテレーション 6760 || Loss: 3.6963 || 10iter: 25.1408 sec.
イテレーション 6770 || Loss: 3.5399 || 10iter: 25.6815 sec.
イテレーション 6780 || Loss: 3.9431 || 10iter: 25.5042 sec.
イテレーション 6790 || Loss: 3.4720 || 10iter: 25.2651 sec.
イテレーション 6800 || Loss: 3.9485 || 10iter: 25.2051 sec.
-------------------------
epoch 38 || EPOCH_TRAIN_Loss : 645.7710 || EPOCH_VAL_Loss : 0.0000
timer 

イテレーション 7930 || Loss: 3.5962 || 10iter: 25.2284 sec.
イテレーション 7940 || Loss: 3.4622 || 10iter: 25.3099 sec.
イテレーション 7950 || Loss: 3.2585 || 10iter: 25.3040 sec.
イテレーション 7960 || Loss: 3.6943 || 10iter: 25.3190 sec.
イテレーション 7970 || Loss: 3.1641 || 10iter: 24.9644 sec.
イテレーション 7980 || Loss: 2.9033 || 10iter: 25.2319 sec.
イテレーション 7990 || Loss: 3.3593 || 10iter: 25.0443 sec.
イテレーション 8000 || Loss: 3.3215 || 10iter: 25.4453 sec.
イテレーション 8010 || Loss: 3.6772 || 10iter: 25.4310 sec.
イテレーション 8020 || Loss: 3.2479 || 10iter: 25.2786 sec.
イテレーション 8030 || Loss: 3.6273 || 10iter: 24.7483 sec.
イテレーション 8040 || Loss: 3.1626 || 10iter: 25.1703 sec.
イテレーション 8050 || Loss: 3.3961 || 10iter: 25.3656 sec.
-------------------------
epoch 45 || EPOCH_TRAIN_Loss : 618.2907 || EPOCH_VAL_Loss : 0.0000
timer : 475.2243 sec
-------------------------
Epoch 46/50
-------------------------
  (train)  
イテレーション 8060 || Loss: 3.4101 || 10iter: 11.9281 sec.
イテレーション 8070 || Loss: 3.3488 || 10iter: 25.2998 sec.
イテレーション 8080 ||

In [ ]:
# これ学習できてんのか？
# lossさほど減っていないような気がする